# Mel-Spectrograms

In this notebook, I go through the process of creating mel-spectrograms from the dataset to train deep learning models. 

Since melspectrograms are numpy arrays, it turns out I can't simply export as csv since numpy arrays are Python-specific objects and they automatically get recognized as string objects when exported as csv. In order to circumvent this issue, I need to save numpy arrays separately.

In [ ]:
# Imports

import numpy as np
import pandas as pd
from glob import glob
import os
import librosa

In [ ]:
# Attach google drive

from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## Function that loops through each genre folder and then compiles all the audio file names

def compile_audio_files(genre_list):
    
    compiled_list = []
    
    for genre in genre_list:
        audio_files = os.listdir(f'/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/{genre}')
        compiled_list += audio_files
        
    return compiled_list

In [ ]:
# Loop through audio files in each genre and add the audio file name to the list

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

final_list = compile_audio_files(genres)

new_final_list = []

for file in final_list:
  variable = file.split('.')[0]
  new_final_list.append(f'/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/{variable}/{file}')

new_final_list[:20] # Final list of music audio files directory

['/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00096.wav_chunk9.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00097.wav_chunk0.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00033.wav_chunk9.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00055.wav_chunk8.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00052.wav_chunk2.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00091.wav_chunk7.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00043.wav_chunk6.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00019.wav_chunk8.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00094.wav_chunk6.wav',
 '/content/gdrive/MyDrive/Music-Genre-Project/genres_3s_split/blues/blues.00087.wav_chunk8.wav',
 '/content/gdrive/MyDrive/Musi

In [ ]:
# Final check
len(new_final_list)

10000

In [ ]:
# Function that extracts melspectrograms and saves as a dataframe

def extract_melspec(files):
    
    melspec = []
    file_name = []
    
    n_fft = 2048 # Number of Short-Time Fourier Transform windows
    hop_length = 512 # How much to shift after each Fourier transform
    n_mels = 128 # Number of separation of frequencies
    
    for file in files:
        y, sr = librosa.load(file)
        y = librosa.feature.melspectrogram(y, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        y = librosa.power_to_db(y, ref=np.max)
        
        if y.shape[1] == 130: # To avoid reshaping issue, we have to drop a few instances where the shape is slightly off
            melspec.append(y)
            file_name.append(file)
        else:
            pass
        
    dict_melspec = {
        'melspec' : melspec,
        'file_name' : file_name
    }

    return pd.DataFrame(dict_melspec)

In [ ]:
# Save the dataframe as a variable
df = extract_melspec(new_final_list)

In [ ]:
# Function that labels genres

def genre_name(file):
    if 'blues' in file:
        return 'blues'
    elif 'classical' in file:
        return 'classical'
    elif 'country' in file:
        return 'country'
    elif 'disco' in file:
        return 'disco'
    elif 'hiphop' in file:
        return 'hiphop'
    elif 'jazz' in file:
        return 'jazz'
    elif 'metal' in file:
        return 'metal'
    elif 'pop' in file:
        return 'pop'
    elif 'reggae' in file:
        return 'reggae'
    elif 'rock' in file:
        return 'rock'

In [ ]:
# Assign genre to each row

df['genre'] = df['file_name'].apply(genre_name)

In [ ]:
# Dictionary that categorizes genres in numbers.
# Not ordinal values. Simply for classification purposes.

genre_num_dict = {
    'blues' : 0,
    'classical' : 1,
    'country' : 2,
    'disco' : 3,
    'hiphop' : 4,
    'jazz' : 5,
    'metal' : 6,
    'pop' : 7,
    'reggae' : 8,
    'rock' : 9
}

In [ ]:
# Assign numbers to genres (no specific order, only for classification purpose)

df['genre_num'] = df['genre'].map(genre_num_dict)

In [ ]:
# Check

df.head()

,melspec,file_name,genre,genre_num
0,"[[-21.381990674164236, -26.59064856269263, -32...",/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
1,"[[-24.882997729703842, -29.4209843264114, -37....",/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
2,"[[-36.865174054353496, -42.8580202940706, -73....",/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
3,"[[-23.014353611424642, -24.69549568481796, -26...",/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
4,"[[-26.309496111579065, -29.813054506919347, -2...",/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0


In [ ]:
# Save the melspec numpy array for later use
np.save('data.npy', np.array(df['melspec']), allow_pickle=True)

In [ ]:
# Demonstration of how to load it for later use
loaded_np = np.load('/content/gdrive/MyDrive/Music-Genre-Project/melspecs/data.npy', allow_pickle=True)

In [ ]:
df_export = df.drop(columns=['melspec'])
df_export.head()

,file_name,genre,genre_num
0,/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
1,/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
2,/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
3,/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0
4,/content/gdrive/MyDrive/Music-Genre-Project/ge...,blues,0


In [ ]:
df_export.to_csv('/content/gdrive/MyDrive/Music-Genre-Project/melspecs/df.csv', index=False)

In [ ]:
# Function to save numpy array and rest of other columns seperately and then brings them together for later use
# https://stackoverflow.com/questions/28439701/how-to-save-and-load-numpy-array-data-properly

def create_df(np_path, df_path):

  loaded_np = np.load(np_path, allow_pickle=True) # Load numpy arrays
  df_exported = pd.read_csv(df_path) # Read in rest of dataframe
  df_exported['melspecs'] = loaded_np # Merge

  return df_exported